# Step 1: El simple y viejo Train / Test

In [1]:
# Limpiamos el entorno
rm(list = ls())
gc(verbose = FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,627604,33.6,1404600,75.1,994249,53.1
Vcells,1149687,8.8,8388608,64.0,1769393,13.5


In [7]:
install.packages("caret")

Installing package into 'C:/Users/German/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)

also installing the dependencies 'listenv', 'parallelly', 'future', 'globals', 'shape', 'future.apply', 'numDeriv', 'progressr', 'SQUAREM', 'diagram', 'lava', 'tzdb', 'prodlim', 'timechange', 'proxy', 'iterators', 'dplyr', 'clock', 'generics', 'gower', 'hardhat', 'ipred', 'lubridate', 'tidyr', 'tidyselect', 'timeDate', 'e1071', 'foreach', 'ModelMetrics', 'plyr', 'pROC', 'recipes', 'reshape2'




package 'listenv' successfully unpacked and MD5 sums checked
package 'parallelly' successfully unpacked and MD5 sums checked
package 'future' successfully unpacked and MD5 sums checked
package 'globals' successfully unpacked and MD5 sums checked
package 'shape' successfully unpacked and MD5 sums checked
package 'future.apply' successfully unpacked and MD5 sums checked
package 'numDeriv' successfully unpacked and MD5 sums checked
package 'progressr' successfully unpacked and MD5 sums checked
package 'SQUAREM' successfully unpacked and MD5 sums checked
package 'diagram' successfully unpacked and MD5 sums checked
package 'lava' successfully unpacked and MD5 sums checked
package 'tzdb' successfully unpacked and MD5 sums checked
package 'prodlim' successfully unpacked and MD5 sums checked
package 'timechange' successfully unpacked and MD5 sums checked
package 'proxy' successfully unpacked and MD5 sums checked
package 'iterators' successfully unpacked and MD5 sums checked
package 'dplyr' suc

In [8]:
# Librerías necesarias
require("data.table")
require("rpart")
require("ROCR")
require("ggplot2")
require("caret")

Loading required package: caret



Loading required package: lattice



In [3]:
# Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:/Users/German/Desktop/dmeyf2023/German/Clase 1") # Establezco el Working Directory

# Poner sus semillas
semillas <- c(279511, 279523, 279541, 279551, 279571)

# Cargamos el dataset
dataset <- fread("competencia_01.csv")
dtrain <- dataset[foto_mes == 202103]

In [4]:
# Nos quedamos solo con el 202101
dataset <- dataset[foto_mes == 202103]
# Creamos una clase binaria
dataset[, clase_binaria := ifelse(
                            clase_ternaria == "BAJA+2",
                                "evento",
                                "noevento"
                            )]
# Borramos el target viejo
dataset[, clase_ternaria := NULL]

In [5]:
# Seteamos nuestra primera semilla
set.seed(semillas[1])

In [9]:
# Particionamos de forma estratificada
in_training <- caret::createDataPartition(dataset$clase_binaria,
                     p = 0.70, list = FALSE) 
dtrain  <-  dataset[in_training, ]
dtest   <-  dataset[-in_training, ]

 Preguntas
 - ¿Por qué separamos en train/test?
 - Son números aleatorios los que nos dan las computadoras
 - ¿Por qué usamos semillas?
 - ¿Qué es una partición estratificada?
 - ¿Tiene realemente alguna ventaja la partición estratificada ?

# Step 2: Armando el primer modelo particionado

In [10]:
# Medimos cuanto tarda nuestro modelo en ajustar
start_time <- Sys.time()
modelo <- rpart(clase_binaria ~ .,
                data = dtrain,
                xval = 0,
                cp = 0,
                minsplit = 20,
                minbucket = 1,
                maxdepth = 5)
print(Sys.time() - start_time)

Time difference of 4.437871 secs


In [11]:
pred_testing <- predict(modelo, dtest, type = "prob")

 Preguntas:
 - ¿Qué tan importante mirar las métricas de train?

# Step 3: Mirando la ganancia

# Step 4: Probando más muchas más semillas

# Step 5: Analizando el azar de las semillas

# Step 6: Tratando de corregir la dispersión

# Step 7: Midiendo nuestras semillas

# Step 8: Buscando un mejor modelo

# TAREA: